# Supplementary Notebook S06: Stoichiometric Model of WL-Pathway with Bifurcating Enzymes

This notebook is supplementary to the the research paper titled:

Optimizing acetogenic CO(2) utilization through metabolic and process engineering highlights the potential of thermophilic bioproduction

In this notebook, similarly to S05, a stoichiometric model of the Wood-Ljungdahl pathway is created, using an existing Genome-Scale Metabolic Model of Clostridium ljungdahlii. Bifurcating enzymes restoring the energy and redox co-factors are this time included, only ATP can be produced or consumed as a sink, along with ADP and Pi. 

In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite

In [2]:
Cljung_Model = cobra.io.read_sbml_model('../../Models/PublishedModels/C-ljungdahlii_iHN637.xml')

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


To simplify the model, 2 changes are made:

The ATP maintenance reaction is unconstrained, so that a flux is not forced for non-growth associated maintenance

The reaction MTHFD2 uses NADPH as a cofactor, to simplify the model, this cofactor is replaced by NADH. This way there is only one

In [3]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

Unconstrain reactions that have been diasbled in the C.ljungdahlii model

In [4]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.HYDFDN.bounds = -1000.0, 1000.0

In [5]:
Cljung_Model.reactions.H2td

Reaction identifier,H2td
Name,Hydrogen transport
Memory address,0x07f9a2d1827f0
Stoichiometry,h2_c <=> h2_e Hydrogen <=> Hydrogen
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport & Exchange:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.EX_ac_e,
Cljung_Model.reactions.EX_etoh_e,
Cljung_Model.reactions.EX_btd_RR_e,
Cljung_Model.reactions.EX_but_e,
Cljung_Model.reactions.EX_btoh_e,
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt2r,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,

Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,

Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.FRNDPR2r,
Cljung_Model.reactions.RNF,
Cljung_Model.reactions.ATPS4r,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,
    
Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

WLmodel.add_boundary(WLmodel.metabolites.get_by_id("atp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("adp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("pi_c"), type="sink")

Reaction identifier,SK_pi_c
Name,Phosphate sink
Memory address,0x07f9a2d2e1c50
Stoichiometry,pi_c <=> Phosphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Add Acetone production pathway

In [7]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [8]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [9]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    #WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [17]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

{}
act_c --> act_e


In [11]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07f9a2d395a90
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

## Change medium, run FBA, and visualize in Escher

In [13]:
medium = WLmodelActn.medium

medium['EX_co2_e'] = 1000
medium['EX_co_e'] = 0
medium['EX_ac_e'] = 0
medium['EX_etoh_e'] = 0
medium['EX_btd_RR_e'] = 0
medium['EX_but_e'] = 0
medium['EX_btoh_e'] = 0
medium['EX_h2_e'] = 10

WLmodelActn.medium = medium

WLmodelActn.medium

{'EX_h2_e': 10,
 'EX_h2o_e': 1000.0,
 'EX_co2_e': 1000,
 'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0}

In [14]:
WLmodelActn.reactions.CODH_ACS.bounds = 1,1

WLmodelActn.objective = "SK_atp_c"
solution = WLmodelActn.optimize()
WLmodelActn.optimize()

,fluxes,reduced_costs
EX_h2_e,-4.000000,-0.000000
EX_h2o_e,2.333333,0.000000
EX_co2_e,-2.000000,-0.000000
EX_co_e,0.000000,-0.666667
EX_pi_e,0.000000,-0.000000
...,...,...
SK_pi_c,-0.333333,-0.000000
ctfAB_ac,0.000000,-2.000000
ADC,0.000000,0.000000
ACTt,0.000000,0.000000


In [15]:
Builder(model = WLmodelActn, map_json = '../../Models/EscherMaps/WLPathway_Bifurcating.json', reaction_data = solution.fluxes.to_dict())


Builder(reaction_data={'EX_h2_e': -4.0, 'EX_h2o_e': 2.3333333333333335, 'EX_co2_e': -2.0, 'EX_co_e': 0.0, 'EX_…

In [1]:
cobra.io.write_sbml_model(WLmodelActn, "../Models/WLPathway_ActnProd_sinks.xml")

NameError: name 'cobra' is not defined